In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
"""
# T15 Practise Solution

def read_data(spark, customSchema):
    '''
    spark_session: spark
    customSchema : we have given the custom schema
    '''
    print("--------------------")
    print ("Starting read_data")
    print("--------------------")
    
    #Mention the Bucket name inside the bucket name variable
    bucket_name = "loan-data926529466287266"
    s3_input_ path = "s3://" + bucket_name + "/inputfile/loan_data.csv"
    df = spark.read.csv(s3_input_path, header=True, schema=customSchema)
    
    return df


def clean_data(input_df):
    '''
    for input file: input_df is output of read_data function
    '''
    print("--------------------")
    print("Starting clean_data")
    print("--------------------")
    
    df = input_df.dropna().dropDuplicates()
    df = df.filter(df.purpose!='null')
    
    return df


def s3_load_data(data, file_name):
    '''
    data : the output data of refult_1 and result_2 function 
    file name : the name of the output to be stored inside the s3
    '''
    #Mention the bucket name inside the bucket_name variable
    bucket_name = "loan-data926529466287266"	
    output_path = "s3://" + bucket_name + "/output"+ file_name
    #output_path = "s3://" + bucket_name + "/output/"+ file_name
    
    if data.count() !=0:
        print("Loading the data", output_path)
        #write the s3 load data command here
        data.coalesce(1).write.csv(output_path, header=True, mode="overwrite")
    
    else:
        print("Empty dataframe, hence cannot save the data", output_path)


def result_1(input_df):
    '''
    for input file: input_df is output of clean_data function
    '''
    print("--------------------------")
    print("Starting result_1")
    print("--------------------------")
    
    df = input_df.filter((col("purpose")=="educational")|(col("purpose")=="small_business"))
    df = df.withColumn("income_to_installment_ratio", col("log_annual_inc")/col("installment"))
    df = df.withColumn("int_rate_category",
                       when(col("int_rate")<0.1, "low")
                       .when((col("int_rate") >= 0.1) & (col("int_rate") < 0.15), "medium" )
                       .otherwise ("high")
                       )
    
    df = df.withColumn("high_risk_borrower",
                      when((col("dti") > 20) | (col("fico") < 700) | (col("revol_util") > 80), 1) # Corrected syntax
                      .otherwise(0)
                      )
    
    return df


def result_2(input_df):
    '''
    for input file: input_df is output of clean_data function
    '''
    print("--------------------------")
    print ("Starting result_2")
    print("--------------------------")
    
    df = input_df.groupBy("purpose").agg(
         (sum(col("not_fully_paid")) / count("*")).alias("default_rate")
         )
    df = df.withColumn("default_rate", round(col("default_rate"), 2))
    
    return df


def redshift_load_data(data):
    if data.count() != 0:
        print("Loading the data into Redshift...")
        jdbcUrl = "jdbc:redshift://emr-spark-redshift.cjgnpeot7x5i.us-east-1.redshift.amazonaws.com:5439/dev"
        username = "awsuser" #Mention redshift username
        password = "Awsuser1" #Mention redshift password
        table_name = "result_2" #Mention redshift table name
    
        #Write the redshift load data command
        data.write \
            .format("jdbc") \
            .option("url", jdbcUrl) \
            .option("dbtable", table_name) \
            .option("user", username) \
            .option("password", password) \
            .mode("overwrite") \
            .save()
    
    else:
        print("Empty dataframe, hence cannot load the data")
"""

'\n# T15 Practise Solution\n\ndef read_data(spark, customSchema):\n    \'\'\'\n    spark_session: spark\n    customSchema : we have given the custom schema\n    \'\'\'\n    print("--------------------")\n    print ("Starting read_data")\n    print("--------------------")\n    \n    #Mention the Bucket name inside the bucket name variable\n    bucket_name = "loan-data926529466287266"\n    s3_input_ path = "s3://" + bucket_name + "/inputfile/loan_data.csv"\n    df = spark.read.csv(s3_input_path, header=True, schema=customSchema)\n    \n    return df\n\n\ndef clean_data(input_df):\n    \'\'\'\n    for input file: input_df is output of read_data function\n    \'\'\'\n    print("--------------------")\n    print("Starting clean_data")\n    print("--------------------")\n    \n    df = input_df.dropna().dropDuplicates()\n    df = df.filter(df.purpose!=\'null\')\n    \n    return df\n\n\ndef s3_load_data(data, file_name):\n    \'\'\'\n    data : the output data of refult_1 and result_2 functi

In [3]:
"""
#### Import statements here
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import resample
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report 

import warnings
import boto3
from sagemaker import get_execution_role

warnings.filterwarnings('ignore')
####



#### Import the dataset from S3
bucket="loan-data602607864436400"
folder_name = "loan_cleaned_data"
data_key = "loan_cleaned_data.csv"
data_location = f's3://{bucket}/{folder_name}/{data_key}'


data = pd.read_csv(data_location)
data.head()


data = pd.get_dummies(data,columns=['purpose'], dtype=int)
data.head()


df_majority=data[data['not_fully_paid']==0]
df_minority=data[data['not_fully_paid']==1]


# Handle the imbalanced data using resample method and oversample the minority class
df_minority_upsampled = resample(df_minority,
                                 replace=True, 
                                 n_samples = df_majority.shape[0],
                                 random_state=42)



# Concatenate the upsampled data records with the majority class records and shuffle the resultant dataframe
df_balanced=pd.concat([df_majority,df_minority_upsampled])
print(df_balanced['not_fully_paid'].value_counts())


# Create X and y data for train-test split
from sklearn.model_selection import train_test_split
from sklearn.ensemble import  RandomForestClassifier

X = df_balanced.drop(columns=['sl_no','not_fully_paid'])
y = df_balanced['not_fully_paid']


# Split the data 
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.4, random_state=42)



# Train a Random Forest Classifier model
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)



# Predict using the trained Random Forest Classifier model
from sklearn.metrics import classification_report
y_pred = rf.predict(X_test)



# Print the classification report 
print(classification_report(y_test, y_pred))



import tempfile

import joblib
BUCKET_NAME ="loan-data602607864436400"
with tempfile.NamedTemporaryFile() as tmp:
    joblib.dump(rf, tmp.name)
    tmp.flush()

    s3= boto3.client('s3')
    
    s3.upload_file(tmp.name, BUCKET_NAME , "model.pkl")

"""

'\n#### Import statements here\nimport pandas as pd\nimport matplotlib.pyplot as plt\nfrom sklearn.utils import resample\nfrom sklearn.utils import shuffle\nfrom sklearn.model_selection import train_test_split \nfrom sklearn.ensemble import RandomForestClassifier\nfrom sklearn.metrics import confusion_matrix, classification_report \n\nimport warnings\nimport boto3\nfrom sagemaker import get_execution_role\n\nwarnings.filterwarnings(\'ignore\')\n####\n\n\n\n#### Import the dataset from S3\nbucket="loan-data602607864436400"\nfolder_name = "loan_cleaned_data"\ndata_key = "loan_cleaned_data.csv"\ndata_location = f\'s3://{bucket}/{folder_name}/{data_key}\'\n\n\ndata = pd.read_csv(data_location)\ndata.head()\n\n\ndata = pd.get_dummies(data,columns=[\'purpose\'], dtype=int)\ndata.head()\n\n\ndf_majority=data[data[\'not_fully_paid\']==0]\ndf_minority=data[data[\'not_fully_paid\']==1]\n\n\n# Handle the imbalanced data using resample method and oversample the minority class\ndf_minority_upsample